In [1]:
"""
Author (in alphabetical order): Hongjian Su, Rui Li, Yingzheng Wang, Zehua Zhou
Date: 12/22/2020
"""

import keras
import h5py
import cv2
import numpy as np

In [2]:
"""
DataLoader class is inspired by https://github.com/csaw-hackml/CSAW-HackML-2020/blob/master/eval.py
"""

class DataLoader:
  def __init__(self, file_path):
    self.file_path = file_path

  def load(self):
    data = h5py.File(self.file_path, "r")
    x_data = np.array(data["data"])
    self.x = x_data.transpose((0,2,3,1))
    self.y = np.array(data["label"])
  
  def preprocess(self):
    self.x = np.asarray(self.x/255, np.float64)

In [3]:
clean_data_validation_filename = "drive/MyDrive/MlForCyberProject/clean_validation_data.h5"
sunglasses_bd_model_filename = "drive/MyDrive/MlForCyberProject/sunglasses_bd_net.h5"
multi_bd_model_filename = "drive/MyDrive/MlForCyberProject/multi_trigger_multi_target_bd_net.h5"
anonymous1_bd_model_filename = "drive/MyDrive/MlForCyberProject/anonymous_1_bd_net.h5"
anonymous2_bd_model_filename = "drive/MyDrive/MlForCyberProject/anonymous_2_bd_net.h5"

validation_clean = DataLoader(clean_data_validation_filename)

validation_clean.load()
validation_clean.preprocess()

sunglasses_bd_model = keras.models.load_model(sunglasses_bd_model_filename)
multi_bd_model = keras.models.load_model(multi_bd_model_filename)
anonymous1_bd_model = keras.models.load_model(anonymous1_bd_model_filename)
anonymous2_bd_model = keras.models.load_model(anonymous2_bd_model_filename)

In [4]:
class STRIP:
  def __init__(self, N, alpha, beta, gamma):
    self.N = N
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma

  def blend_image(self, img1, img2):
    return cv2.addWeighted(img1,self.alpha,img2,self.beta,self.gamma).reshape(55,47,3)

  def calculate_entropy(self, target_img, model, input_img):
    blended_img = np.zeros((self.N,55,47,3))
    random_index = np.random.choice(np.arange(len(input_img)), self.N, replace=False)
    for i in range(self.N):
      blended_img[i] = self.blend_image(target_img, input_img[random_index[i]])
    pred_label = model.predict(blended_img)
    entropy = -np.nansum(pred_label*np.log2(pred_label))
    return entropy

  def generate_entropy_distribution(self, input_img, clean_img, model):
    l = len(input_img)
    entropy_distribution = np.zeros(l)
    for i in range(l):
      target_img = input_img[i]
      entropy_distribution[i] = self.calculate_entropy(target_img, model, clean_img)
    return entropy_distribution

In [5]:
STRIP_filter = STRIP(50, 1.5, 1, 0)

entropy_clean_sunglasses = STRIP_filter.generate_entropy_distribution(validation_clean.x, validation_clean.x, sunglasses_bd_model)
entropy_clean_multi = STRIP_filter.generate_entropy_distribution(validation_clean.x, validation_clean.x, multi_bd_model)
entropy_clean_anonymous1 = STRIP_filter.generate_entropy_distribution(validation_clean.x, validation_clean.x, anonymous1_bd_model)
entropy_clean_anonymous2 = STRIP_filter.generate_entropy_distribution(validation_clean.x, validation_clean.x, anonymous2_bd_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


In [6]:
f_sunglasses = h5py.File("entropy_clean_sunglasses.h5", "w")
f_sunglasses["data"] = entropy_clean_sunglasses
f_sunglasses.close()
f_multi = h5py.File("entropy_clean_multi.h5", "w")
f_multi["data"] = entropy_clean_multi
f_multi.close()
f_anonymous1 = h5py.File("entropy_clean_anonymous1.h5", "w")
f_anonymous1["data"] = entropy_clean_anonymous1
f_anonymous1.close()
f_anonymous2 = h5py.File("entropy_clean_anonymous2.h5", "w")
f_anonymous2["data"] = entropy_clean_anonymous2
f_anonymous2.close()